In [1]:
from mainnet_launch.pages.risk_metrics.estimate_exit_liquidity_from_quotes import *
from mainnet_launch.constants import *

chain, base_asset, valid_autopools = (
    ETH_CHAIN,
    WETH,
    [a for a in ALL_AUTOPOOLS if a.chain == ETH_CHAIN and a.base_asset == WETH(ETH_CHAIN)],
)

unscaled_asset_exposure, percent_ownership_by_destination_df, token_df = fetch_needed_context(chain, valid_autopools)

tokemak_quote_requests_df, odos_quote_requests_df, raw_tokemak_quote_response_df, raw_odos_quote_response_df = (
    fetch_odos_and_tokemak_quotes(chain=chain, base_asset=base_asset, valid_autopools=valid_autopools)
)

https://swaps-pricing.tokemaklabs.com/swap-quote-v2


Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 5/5 [00:03<00:00,  1.33it/s]


https://api.odos.xyz/sor/quote/v2


Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 5/5 [00:03<00:00,  1.35it/s]


https://swaps-pricing.tokemaklabs.com/swap-quote-v2


...

Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 5/5 [00:05<00:00,  1.14s/it]


https://swaps-pricing.tokemaklabs.com/swap-quote-v2


Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


https://api.odos.xyz/sor/quote/v2


Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 5/5 [00:02<00:00,  1.70it/s]


In [ ]:
@dataclass
class QuoteResponse:
    # I think this should be the table in the database
    api: str
    chain_id: int

    token_in: str
    token_out: str
    token_in_symbol: str
    token_out_symbol: str  # technically redundent, but useful for display
    scaled_amount_in: float
    scaled_amount_out: float

    datetime_received: pd.Timestamp

    pools_blacklist: tuple[str]  # empty for tokemak,
    aggregator_name: str  # odos for odos, else aggregator name for tokemak


def _post_process_raw_tokemak_quote_response_df(
    raw_tokemak_quote_response_df: pd.DataFrame, token_df: pd.DataFrame
) -> list[QuoteResponse]:
    token_to_decimal = token_df.set_index("token_address")["decimals"].to_dict()
    token_to_symbols = token_df.set_index("token_address")["symbol"].to_dict()

    tokemak_quote_responses = [
        QuoteResponse(
            api="tokemak",
            chain_id=int(row["chainId"]),
            token_in=row["sellToken"],
            token_out=row["buyToken"],
            token_in_symbol=token_to_symbols[row["sellToken"]],
            token_out_symbol=token_to_symbols[row["buyToken"]],
            scaled_amount_in=int(row["sellAmount"]) / 10 ** token_to_decimal[row["sellToken"]],
            scaled_amount_out=int(row["buyAmount"]) / 10 ** token_to_decimal[row["buyToken"]],
            datetime_received=row["datetime_received"],
            pools_blacklist=(),  # tokemak can't blacklist pools, so we use an empty tuple
            aggregator_name=row["aggregatorName"],
        )
        for _, row in raw_tokemak_quote_response_df.iterrows()
    ]

    return tokemak_quote_responses


def _post_process_raw_odos_quote_response_df(
    raw_odos_quote_response_df: pd.DataFrame, token_df: pd.DataFrame
) -> list[QuoteResponse]:
    token_to_decimal = token_df.set_index("token_address")["decimals"].to_dict()
    token_to_symbols = token_df.set_index("token_address")["symbol"].to_dict()

    odos_quote_responses: list[QuoteResponse] = []
    for _, row in raw_odos_quote_response_df.iterrows():

        token_in = Web3.toChecksumAddress(row["inTokens"])
        token_out = Web3.toChecksumAddress(row["outTokens"])

        unscaled_amount_in = int(row["inAmounts"])
        unscaled_amount_out = int(row["outAmounts"])

        decimals_token_in = token_to_decimal[token_in]
        decimals_token_out = token_to_decimal[token_out]

        quote_response = QuoteResponse(
            api="odos",
            chain_id=int(row["chainId"]),
            token_in=token_in,
            token_out=token_out,
            token_in_symbol=token_to_symbols[token_in],
            token_out_symbol=token_to_symbols[token_out],
            scaled_amount_in=unscaled_amount_in / 10**decimals_token_in,
            scaled_amount_out=unscaled_amount_out / 10**decimals_token_out,
            datetime_received=row["datetime_received"],
            pools_blacklist=tuple(row["poolBlacklist"]),
            aggregator_name="Odos",
        )
        odos_quote_responses.append(quote_response)

    return odos_quote_responses


def _clean_responses(
    raw_odos_quote_response_df: pd.DataFrame, raw_tokemak_quote_response_df: pd.DataFrame, token_df: pd.DataFrame
) -> pd.DataFrame:

    cleaned_odos_responses = _post_process_raw_odos_quote_response_df(raw_odos_quote_response_df, token_df)
    clean_odos_response_df = pd.DataFrame(cleaned_odos_responses)
    cleaned_tokemak_responses = _post_process_raw_tokemak_quote_response_df(raw_tokemak_quote_response_df, token_df)
    clean_tokemak_response_df = pd.DataFrame(cleaned_tokemak_responses)

    df = pd.concat([clean_odos_response_df, clean_tokemak_response_df], ignore_index=True)

    df["effective_price"] = df["scaled_amount_out"] / df["scaled_amount_in"]

    return df

,api,chain_id,token_in,token_out,token_in_symbol,token_out_symbol,scaled_amount_in,scaled_amount_out,datetime_received,pools_blacklist,aggregator_name,effective_price
0,odos,1,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,stETH,WETH,100.0,99.868111,2025-08-07 18:46:41.560941+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.998681
1,odos,1,0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,osETH,WETH,20.0,21.032431,2025-08-07 18:46:41.561891+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.051622
2,odos,1,0xae78736Cd615f374D3085123A210448E74Fc6393,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,rETH,WETH,200.0,227.581867,2025-08-07 18:46:42.076729+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.137909
3,odos,1,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,ezETH,WETH,50.0,52.742307,2025-08-07 18:46:42.954250+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.054846
4,odos,1,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,pxETH,WETH,20.0,19.925611,2025-08-07 18:46:44.314680+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.996281
5,odos,1,0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,osETH,WETH,20.0,21.032431,2025-08-07 18:47:18.297853+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.051622
6,odos,1,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,stETH,WETH,100.0,99.868076,2025-08-07 18:47:18.318183+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.998681
7,odos,1,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,ezETH,WETH,50.0,52.742307,2025-08-07 18:47:18.393928+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.054846
8,odos,1,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,pxETH,WETH,20.0,19.925611,2025-08-07 18:47:20.570160+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.996281
9,odos,1,0xae78736Cd615f374D3085123A210448E74Fc6393,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,rETH,WETH,200.0,227.581869,2025-08-07 18:47:22.979036+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.137909


In [13]:
px.scatter(
    df,
    x="scaled_amount_in",
    y="effective_price",
    color="token_in_symbol",
)

,api,chain_id,token_in,token_out,token_in_symbol,token_out_symbol,scaled_amount_in,scaled_amount_out,datetime_received,pools_blacklist,aggregator_name,effective_price
0,odos,1,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,stETH,WETH,100.0,99.868111,2025-08-07 18:46:41.560941+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.998681
1,odos,1,0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,osETH,WETH,20.0,21.032431,2025-08-07 18:46:41.561891+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.051622
2,odos,1,0xae78736Cd615f374D3085123A210448E74Fc6393,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,rETH,WETH,200.0,227.581867,2025-08-07 18:46:42.076729+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.137909
3,odos,1,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,ezETH,WETH,50.0,52.742307,2025-08-07 18:46:42.954250+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.054846
4,odos,1,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,pxETH,WETH,20.0,19.925611,2025-08-07 18:46:44.314680+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.996281
5,odos,1,0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,osETH,WETH,20.0,21.032431,2025-08-07 18:47:18.297853+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.051622
6,odos,1,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,stETH,WETH,100.0,99.868076,2025-08-07 18:47:18.318183+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.998681
7,odos,1,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,ezETH,WETH,50.0,52.742307,2025-08-07 18:47:18.393928+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.054846
8,odos,1,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,pxETH,WETH,20.0,19.925611,2025-08-07 18:47:20.570160+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,0.996281
9,odos,1,0xae78736Cd615f374D3085123A210448E74Fc6393,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,rETH,WETH,200.0,227.581869,2025-08-07 18:47:22.979036+00:00,"(0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5, 0...",Odos,1.137909


In [9]:
df.pivot_table(
    columns="token_in_symbol",
    index="scaled_amount_in",
    values="effective_price",
)

token_in_symbol,ezETH,osETH,pxETH,rETH,stETH
scaled_amount_in,,,,,
20.0,NaN,1.05172,0.996327,NaN,NaN
50.0,1.054846,NaN,NaN,NaN,NaN
100.0,NaN,NaN,NaN,NaN,0.998681
200.0,NaN,NaN,NaN,1.13791,NaN


In [5]:
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

px.scatter(
    df,
    x="scaled_amount_in",
    y="effective_price",
    color="token_in_symbol",
    hover_data=["api", "datetime_received", "aggregator_name"],
)